In [1]:
%pip install langchain_groq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install groq

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os

llm = ChatGroq(
    model="gemma2-9b-it",
    api_key=os.getenv("groq")
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절하고 유익한 AI 조수입니다. 게임에 대해서 설명하는 AI다"),
    ("human", "{question}")
])
chain = prompt | llm

chain.invoke({"question": "스타크래프트에 대해 설명"}).content



'안녕하세요! 스타크래프트에 대해 알고 싶으시군요! \n\n스타크래프트는 블리자드 엔터테인먼트가 개발한 실시간 전략 시뮬레이션 게임으로, 1998년에 처음 출시되었습니다. \n\n**게임의 핵심은 다음과 같습니다:**\n\n* **3개의 종족:** 테란, 저그, 프로토스 각각의 특징과 강점을 가지고 있습니다.\n    * **테란:** 인간으로, 뛰어난 기술력을 바탕으로 강력한 무기와 방어 시설을 구축합니다.\n    * **저그:** 벌레처럼 생긴 종족으로, 빠른 지능과 군대의 숫자로 적을 압도합니다.\n    * **프로토스:** 고대 문명을 유지하며, 강력한 무기와 기술력을 바탕으로 전투를 이어갑니다.\n* **자원 확보:** 미네랄과 가스를 채굴하여 유닛과 건물을 생산합니다.\n* **유닛 생산 및 전투:** 다양한 유닛을 생산하여 적의 유닛을 공격하고 방어합니다.\n* **기술 개발:** 연구소에서 기술을 개발하여 유닛과 건물을 강화합니다.\n* **맵 지배:** 게임맵을 확보하여 적의 공격을 막고 자신의 영토를 확장합니다.\n\n**스타크래프트의 매력은:**\n\n* **깊이 있는 전략:** 유닛 배치, 기술 개발, 자원 관리 등 다양한 전략적 요소가 있습니다.\n* **빠른 게임 속도:** 게임 속도가 빠르기 때문에 몰입감이 높습니다.\n* **높은 재미 요소:** 승리의 짜릿함과 패배의 아픔을 동시에 느낄 수 있습니다.\n* **다양한 게임 모드:** 싱글플레이, 멀티플레이 등 다양한 게임 모드를 즐길 수 있습니다.\n* **활발한 커뮤니티:** 전 세계적으로 많은 팬들이 있으며, 활발한 커뮤니티가 존재합니다.\n\n스타크래프트는 어려울 수 있지만, 뛰어난 전략과 흥미진진한 게임 플레이를 갖춘 게임입니다. 새로운 플레이어에게는 다양한 가이드와 학습 자료가 있으니 걱정하지 마세요! \n\n더 궁금한 점이 있다면 언제든지 물어보세요!\n'

# 문제점(07.21 기준)

In [7]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os

llm = ChatGroq(
    model="gemma2-9b-it",
    api_key=os.getenv("groq")
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 한국에 대해서 설명하는 AI다"),
    ("human", "{question}")
])
chain = prompt | llm

chain.invoke({"question": "대한민국 대통령은 누구인가?"}).content



'대한민국 현재 대통령은 **윤석열** 입니다. 2022년 5월 10일부터 임기를 시작했습니다.  \n'

In [7]:
%pip install langchain_community duckduckgo-search

/usr/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=7659) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor, create_react_agent
import os

# 1. LLM 초기화
llm = ChatGroq(
    model="gemma2-9b-it", # 또는 gemma-7b-it과 같이 더 안정적인 모델 고려
    api_key=os.getenv("groq")
)

# 2. 웹 검색 도구 정의
tools = [
    DuckDuckGoSearchRun(name="web_search")
]

# 3. 에이전트 프롬프트 정의
# ReAct 에이전트가 필요로 하는 'tools', 'tool_names', 'agent_scratchpad'를
# 시스템 프롬프트에 명시적으로 포함하고, ReAct 패턴을 따르도록 강력하게 지시합니다.
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 한국에 대한 질문에 답변하는 AI입니다. 정확하고 최신 정보를 제공해야 합니다.
질문에 답하기 위해 필요한 경우 웹 검색 도구({tool_names})를 사용하세요.

당신은 다음 형식으로 추론하고 행동해야 합니다:
Thought: (여기서 질문에 답하기 위해 어떤 행동을 해야 할지 추론합니다. 도구를 사용할지, 아니면 최종 답변을 할지 결정합니다.)
Action: (수행할 작업의 이름. 사용 가능한 도구 중 하나여야 합니다: {tool_names})
Action Input: (액션에 대한 입력. 도구의 입력 형식과 일치해야 합니다.)
Observation: (액션의 결과)
... (위 Thought/Action/Action Input/Observation을 필요한 만큼 반복)
Thought: (최종 답변을 할 때가 되었다고 판단합니다.)
Final Answer: (최종 답변)

사용 가능한 도구:
{tools}

{agent_scratchpad}
"""),
    ("human", "{input}"),
])

# 4. ReAct 에이전트 생성
agent = create_react_agent(llm, tools,prompt)

# 5. AgentExecutor 생성 및 실행
# handle_parsing_errors=True를 추가하여 파싱 오류를 에이전트에게 전달합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True)

# 에이전트 실행 예시
result = agent_executor.invoke({"input": "대한민국의 대통령은?"})

print(result['output'])

/home/an0jin/.local/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


대한민국의 현 대통령은 이재명입니다.
